<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Variables" data-toc-modified-id="Variables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Variables</a></span></li><li><span><a href="#Funciones" data-toc-modified-id="Funciones-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Funciones</a></span></li><li><span><a href="#Programa-Ppal" data-toc-modified-id="Programa-Ppal-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Programa Ppal</a></span></li><li><span><a href="#i+d" data-toc-modified-id="i+d-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>i+d</a></span></li></ul></div>

# Variables

In [3]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import datetime
import random


# ahora = datetime.datetime.now()
# ahora.year


######## Codigo PERSONAL###############
proxy_on = False

 
proxies_home = None
proxies_work = {
  "http": "http://uid:pass@ip:port",
  "https": "http://uid:pass@ip:port",
}

if proxy_on:
    proxies = proxies_work
else:
    proxies = proxies_home
########################################

headers={
    "Accept": "*/*",
    "Content-Encoding": "gzip",
    "Cache-Control": "max-age=1206531",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
}

URL = "https://www.atrapalo.com/entradas/"
'''
SECCIONES = {'teatro-y-danza' : 'Teatro y Danza', 
           'musica': 'Música', 
           'parques-tematicos': 'Parques temáticos', 
            'musicales': 'Musicales',
            'museos-y-exposiciones':'Museos y Exposiciones',
            'deportes':'Deportes',
            'circo': 'Circo',
            'cine': 'Cine',
            'feria':'Ferias'}

'''
SECCIONES = {'teatro-y-danza' : 'Teatro y Danza'}



# Funciones

In [4]:
# Desarrollo de funciones

'''
DESCRIPCIÓN: 
Función a la que se le pasa la URL de la web ATRAPALO para que se descargue todos los espectaculos que se listan
en el apartado de 'entradas' 

    INPUT: 
        url (string) : URL de Atrapalo   
    
    OUTPUT: 
        nuevos_espectaculos (list): lista con todos los espectaculos encontrados

''' 

def Sacar_espectaculos(url):
    
    respuesta = requests.get(url, proxies=proxies, headers=headers)
    print("Rta:",respuesta.status_code)
    soup = BeautifulSoup(respuesta.content, "html.parser")
    nuevos_espectaculos = soup.find_all(class_="card-result-container")    
    return(nuevos_espectaculos)
    
    
'''
DESCRIPCIÓN: 
Función a la que se le pasa el código html de cada uno de los espectaculos y devuelve una tupla con los atributos extraidos

    INPUT: 
        espectaculos[N]    donde N corresponde el N-ésimo espectáculo obtenido
    
    OUTPUT: 
        atributos (tupla) donde se entregan todos los atributos extráidos

''' 

def Sacar_atributos(espectaculo):
    
     # Asignamos valores nulos para aquellos registros de los que no podamos sacr la info de la web
    popularidad = ""
    popularidad_desc = ""
    opiniones = ""
    
    try:
        
        # titulo
        titulo= espectaculo.find_all('h2')[0].text.strip('\n')
        
        # Ubicacion
        ubicacion = espectaculo.find_all(class_="info")[0].find('a').text
        
        # fechas en cartes, inicial y final
        num = espectaculo.find_all(class_="info")[1].find_all('span')
        fechas= espectaculo.find_all(class_="info")[1].find_all('span')[0].text
        if len(num) >=3:
            fecha_inicio= espectaculo.find_all(class_="info")[1].find_all('span')[1].span['title']
            fecha_fin= espectaculo.find_all(class_="info")[1].find_all('span')[4]['title']
        else: 
            fecha_inicio = espectaculo.find_all(class_="info")[1].find_all('span')[1]['title']
            fecha_fin = fecha_inicio
    
        # duracion
        duracion = espectaculo.find_all(class_="info")[2].get_text().strip('\n').strip() 
    
        # Idioma
        if len(espectaculo.find_all(class_="info")) == 4:
            idioma = espectaculo.find_all(class_="info")[3].get_text().strip('\n').strip()
        else:
            idioma = "S/R"
        
        
       # Pupularidad
        
        popularidad = espectaculo.find(class_="opi-rating").text.strip('\n').strip()
        popularidad_desc = espectaculo.find(class_="opi-description").a.text.strip('\n').strip()
        
        # Opiniones
        opiniones = espectaculo.find(class_="opi-description").span.text.strip('\n').strip()
        
            
    except AttributeError: 
        
        pass
    
    atributos = (titulo,
                 ubicacion,
                 fechas,
                 fecha_inicio,
                 fecha_fin,
                 duracion,
                 idioma,
                 popularidad,
                 popularidad_desc,
                 opiniones)
    
    return (atributos)

# Programa Ppal

In [5]:
'''
PROGRAMA PRINCIPAL
'''

espectaculos = []
espectaculos_pantalla_anterior = []

for seccion in SECCIONES:
    cont = 1
    seguir_buscando = True
    while seguir_buscando:
        if cont == 0:
            url = URL + seccion + '/'
        else: 
            url = URL + seccion + '/' + 'p-' + str(cont) + '/'
        cont += 1
    
        if cont==6:
            
            seguir_buscando = False
        
        espectaculos_pantalla = Sacar_espectaculos(url)
        print(url,":", len(espectaculos_pantalla))
        
        
        if espectaculos_pantalla == espectaculos_pantalla_anterior:
            seguir_buscando = False
        else:
            
            for espectaculo in espectaculos_pantalla:
                espectaculos.append(espectaculo)
                
            espectaculos_pantalla_anterior = espectaculos_pantalla 
    
        
        # Pendiente de Desarrollo
        #for espectaculo in espectaculos:
            #ahorro = espectaculo.find('span').find('span').string
            #lugar = espectaculo.find_all(class_="").a['title']
    

Rta: 200
https://www.atrapalo.com/entradas/teatro-y-danza/p-1/ : 20
Rta: 200
https://www.atrapalo.com/entradas/teatro-y-danza/p-2/ : 20
Rta: 200
https://www.atrapalo.com/entradas/teatro-y-danza/p-3/ : 20
Rta: 200
https://www.atrapalo.com/entradas/teatro-y-danza/p-4/ : 20
Rta: 200
https://www.atrapalo.com/entradas/teatro-y-danza/p-5/ : 20


# i+d

In [6]:
len(espectaculos)
#Sacar_atributos(espectaculos[1])

100

In [10]:
#i+d
for i in espectaculos:
    print(Sacar_atributos(i))

('Perfectos desconocidos', 'Teatro Reina Victoria', '03 abr al 23 jun', '2019/04/03', '2019/06/23', '90 minutos', 'Castellano', '9.0', 'Extraordinario', '565 opiniones')
('Monólogos de humor - Chocita del Loro Gran vía', 'Teatro Chocita del Loro - Senator', '02 abr al 30 abr', '2019/04/02', '2019/04/30', '1 hora y 30 minutos', 'Castellano', '8.7', 'Excelente', '11530 opiniones')
('Impro Show', 'Teatreneu', '04 abr al 31 may', '2019/04/04', '2019/05/31', '80 minutos', 'Castellano', '8.6', 'Excelente', '5285 opiniones')
('David Guapo - #quenonosfrunjanlafiesta1, en Barcelona', 'Club Capitol', '04 abr al 28 abr', '2019/04/04', '2019/04/28', '1 hora y 20 minutos', 'Español', '9.0', 'Extraordinario', '3264 opiniones')
('La Tendresa', 'Teatre Poliorama', '04 abr al 19 abr', '2019/04/04', '2019/04/19', '110 minutos', 'Catalán', '9.1', 'Extraordinario', '231 opiniones')
('Burundanga', 'Teatro Maravillas', '03 abr al 30 jun', '2019/04/03', '2019/06/30', '90 minutos', 'Español', '8.4', 'Muy buen

UnboundLocalError: local variable 'ubicacion' referenced before assignment

In [275]:
#espectaculos[1].find_all(class_="info")[0].find('a').text
espectaculos[1].find(class_="opi-description").span.text.strip('\n').strip()


'553 opiniones'

In [276]:
for cont,espectaculo in enumerate(espectaculos):
    
    try:
        titulo= espectaculo.find_all('h2')[0].text.strip('\n')
        #ubicacion = espectaculos[0].find_all(class_="info")[0].span.a.text
        #de_a = espectaculos[0].find_all(class_="info")[1].text.strip('\n').strip()
    
        # fechas
        num = espectaculo.find_all(class_="info")[1].find_all('span')
        fechas= espectaculo.find_all(class_="info")[1].find_all('span')[0].text
        if len(num) >=3:
            fecha_inicio= espectaculo.find_all(class_="info")[1].find_all('span')[1].span['title']
            fecha_fin= espectaculo.find_all(class_="info")[1].find_all('span')[4]['title']
        else: 
            fecha_inicio = espectaculo.find_all(class_="info")[1].find_all('span')[1]['title']
            fecha_fin = fecha_inicio
    
        duracion = espectaculo.find_all(class_="info")[2].get_text().strip('\n').strip() 
    
        if len(espectaculo.find_all(class_="info")) == 4:
            idioma = espectaculo.find_all(class_="info")[3].get_text().strip('\n').strip()
        else:
            idioma = "S/R"
        
        
        # atributo = 'Popularidad'
        popularidad = ""
        popularidad_desc = ""
        opiniones = ""
        
        popularidad = espectaculo.find(class_="opi-rating").text.strip('\n').strip()
        popularidad_desc = espectaculo.find(class_="opi-description").a.text.strip('\n').strip()
        opiniones = espectaculo.find(class_="opi-description").span.text.strip('\n').strip()
        
            
    except AttributeError: 
        
        pass
        #if atributo == 'Popularidad': popularidad = ""
    
    print(f"{cont}: {len(num)} :{titulo} | {ubicacion} |  {fechas} |  {fecha_inicio}  |  {fecha_fin} | {duracion} | {idioma} | {popularidad} | {popularidad_desc} | {opiniones}")
    #print(f"{cont}: {len(num)} :{titulo} | {ubicacion} |  {fechas} |  {fecha_inicio}  |  {fecha_fin} | {duracion} | {idioma} | {popularidad}")
    

0: 2 :Adiós Arturo - La Cubana, en Las Palmas | Teatro Cuyás |  

 |  2019/04/04  |  2019/04/04 | 120 minutos aprox | Español |  |  | 
1: 5 :Perfectos desconocidos | Teatro Cuyás |  29 mar al 23 jun |  2019/03/29  |  2019/06/23 | 90 minutos | Castellano | 9.0 | Extraordinario | 553 opiniones
2: 5 :Monólogos de humor - Chocita del Loro Gran vía | Teatro Cuyás |  29 mar al 30 abr |  2019/03/29  |  2019/04/30 | 1 hora y 30 minutos | Castellano | 8.7 | Excelente | 11507 opiniones
3: 5 :Burundanga | Teatro Cuyás |  29 mar al 30 jun |  2019/03/29  |  2019/06/30 | 90 minutos | Español | 8.4 | Muy bueno | 12641 opiniones
4: 5 :Impro Show | Teatro Cuyás |  29 mar al 31 may |  2019/03/29  |  2019/05/31 | 80 minutos | Castellano | 8.6 | Excelente | 5272 opiniones
5: 5 :Nada es imposible - Mago Pop | Teatro Cuyás |  29 mar al 23 jun |  2019/03/29  |  2019/06/23 | 90 minutos | Castellano | 9.1 | Extraordinario | 1832 opiniones
6: 5 :Las noches de El Club de la Comedia, en Madrid | Teatro Cuyás |  3